In [3]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from api_keys_ml import api_key

ModuleNotFoundError: No module named 'api_keys_ml'

In [2]:
nasdaq_url = 'https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey=' + api_key

NameError: name 'api_key' is not defined

In [ ]:
nasdaq_list_response = requests.get(nasdaq_url).json()

In [ ]:
# collect a list of the NASDAQ constituents
nasdaq_companies = []
company_tickers = []

for company in nasdaq_list_response:
    nasdaq_companies.append(company['name'])
    company_tickers.append(company['symbol'])

In [ ]:
data = {'Company': company_tickers, 'Q1 P/E': ['' for i in range(len(company_tickers))], 
        'Q2 P/E': ['' for i in range(len(company_tickers))], 
        'Q3 P/E': ['' for i in range(len(company_tickers))], 
        'Q1 ROE': ['' for i in range(len(company_tickers))], 
        'Q2 ROE': ['' for i in range(len(company_tickers))], 
        'Q3 ROE': ['' for i in range(len(company_tickers))], 
        'Q1 Date': ['' for i in range(len(company_tickers))],
        'Q2 Date': ['' for i in range(len(company_tickers))],
        'Q3 Date': ['' for i in range(len(company_tickers))],
       }
nasdaq_df = pd.DataFrame(data=data)
nasdaq_df.head()

In [ ]:
# gather data on price/earnings ratio AND returns on equity for each company for the last three quarters
for index, row in nasdaq_df.iterrows():
    ticker = row['Company']
    quarterly_ratio_url = f'https://financialmodelingprep.com/api/v3/ratios/{ticker}?period=quarter&limit=3&apikey={api_key}' 
    response = requests.get(quarterly_ratio_url).json()

    try:
        nasdaq_df.loc[index, 'Q3 P/E'] = response[0]['priceEarningsRatio']
        nasdaq_df.loc[index, 'Q2 P/E'] = response[1]['priceEarningsRatio']
        nasdaq_df.loc[index, 'Q1 P/E'] = response[2]['priceEarningsRatio']
        nasdaq_df.loc[index, 'Q3 ROE'] = response[0]['returnOnEquity']
        nasdaq_df.loc[index, 'Q2 ROE'] = response[1]['returnOnEquity']
        nasdaq_df.loc[index, 'Q1 ROE'] = response[2]['returnOnEquity']
        nasdaq_df.loc[index, 'Q3 Date'] = response[0]['date']
        nasdaq_df.loc[index, 'Q2 Date'] = response[1]['date']
        nasdaq_df.loc[index, 'Q1 Date'] = response[2]['date']
    except IndexError:
        print('Missing NASDAQ information')

    
    

In [ ]:
nasdaq_df.head()

In [ ]:
nasdaq_df.to_csv('../output/nasdaq_financial_ratios.csv')

In [ ]:
x_values = nasdaq_df["Q1 ROE"].mean().to_list()